### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1000]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_and_student_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_and_student_df





,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [1001]:
# ----------------------------

##### totals calculations
#groups the data by type, turning it into a groupby object
schooldata_typegroup=school_and_student_df.groupby("type")
#Returns unique students and school names for each type
dist_totals=pd.DataFrame(schooldata_typegroup["school_name","Student ID",].nunique())
#turns the original school data into a grouby by object in the type series
totalbudget_dist=school_data.groupby("type").sum()
## add the total budget from the totalbudget_dist
dist_totals=pd.merge(dist_totals,totalbudget_dist["budget"],on="type")




###### averages calculation
#joins the data on "type", calculating the mean
dist_averages=pd.DataFrame(school_and_student_df.groupby("type").mean())
# deletes unnecessary series in dist_averages
need_to_delete_list=['budget','Student ID','School ID','size']
for derp in need_to_delete_list:
    del dist_averages[derp]


###calculates the percentages--(I could iterate through the reading and math sections of the below code to reduce redundancy)
#calculating math percentage
#pulls in the student id and type if the studen's mathscore is equal to or above
passingmath_dist = school_and_student_df.loc[school_and_student_df["math_score"]>= 70,["Student ID","type"]]
# groups the data by type then turns it into a data frame
passingmath_summary_dist=passingmath_dist.groupby("type").count()
pd.DataFrame(passingmath_summary_dist)
#Calculates the percentage of students passing math and renames the student id column correctly
passingmath_summary_dist["% Passing math"]=passingmath_summary_dist["Student ID"]/dist_totals["Student ID"]
passingmath_summary_dist=passingmath_summary_dist.rename(columns={'Student ID':'Students Passing Math','% Passing math':'% Passing Math'})
#-----
##calculating reading percentage
#pulls in the student id and type if the studen's readingscore is equal to or above 70
passingread_dist = school_and_student_df.loc[school_and_student_df["reading_score"]>= 70,["Student ID","type"]]
# groups the data by type then turns it into a data frame
passingread_summary_dist=passingread_dist.groupby("type").count()
pd.DataFrame(passingread_summary_dist)
#calculates the percentage of students passing reading and renames the student id column correctly
passingread_summary_dist["% Passing read"]=passingread_summary_dist["Student ID"] / dist_totals["Student ID"]
passingread_summary_dist=passingread_summary_dist.rename(columns={'Student ID':'Students Passing Reading','% Passing read':'% Passing Reading'})
#-----
##calculating the overall pass rates
percentpassingoverall_dist=school_and_student_df.loc[((school_and_student_df['math_score']>=70) & (school_and_student_df['reading_score']>=70)),["Student ID","type"]]

# groups the data by type then turns it into a data frame
percentpassingoverall_summary_dist=percentpassingoverall_dist.groupby("type").count()
pd.DataFrame(percentpassingoverall_summary_dist)
#calculates the percentage of students passing math/reading and renames the student id column correctly
percentpassingoverall_summary_dist["% Passing Overall"]=percentpassingoverall_summary_dist["Student ID"] / dist_totals["Student ID"]
percentpassingoverall_summary_dist=percentpassingoverall_summary_dist.rename(columns={'Student ID':'# Passing Overall'})
percentpassingoverall_summary_dist.head()

#-----------------------------------------------
#need to format the table so it looks pretty
#renames the headers of dist_totals
dist_totals=dist_totals.rename(columns={'school_name':'# of schools','Student ID':'# of students'})
#renames the headers of dist_averages
dist_averages=dist_averages.rename(columns={'reading_score':'Average Reading Score','math_score':'Average Math Score'})


#list of gathered data frames excluding dist_totals
summarydf_list = [dist_averages,passingmath_summary_dist,passingread_summary_dist,percentpassingoverall_summary_dist]

#adds dist_totals to the summary dataframe manually
summary_dist_df=dist_totals
#loops through the remaining dataframes and joins them to the summary dataframe
for sublist in summarydf_list:
    summary_dist_df=pd.merge(sublist,summary_dist_df,on="type")



# # need to format the columns here
#eliminate columns # Passing Overall, Students Passing Reading, Students Passing Math,
another_delety_list = ['# Passing Overall', 'Students Passing Reading', 'Students Passing Math']

for each in another_delety_list:
    del summary_dist_df[each]

#reorder based on readme
dist_reorder_list=['# of schools','# of students','budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Passing Overall']
summary_dist_df=summary_dist_df[dist_reorder_list]

# format the 3 percentage columns as percentage
dist_percent_listy_loop=['Reading','Math','Overall']
for percenty in dist_percent_listy_loop:
    summary_dist_df["% Passing "+percenty]=summary_dist_df["% Passing "+percenty].map('{:.2%}'.format)

#rename and forma the budget into currency format
summary_dist_df=summary_dist_df.rename(columns={"budget":"Total Budget"})

summary_dist_df["Total Budget"]=summary_dist_df["Total Budget"].map('${:,.2f}'.format)



summary_dist_df

,# of schools,# of students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
type,,,,,,,,
Charter,8,12194,"$7,301,505.00",83.406183,83.902821,93.70%,96.65%,90.56%
District,7,26976,"$17,347,923.00",76.987026,80.962485,66.52%,80.91%,53.70%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [1002]:
# School Name -
# School Type -
# Total Students -
# Total School Budget -
# Per Student Budget -
# Average Math Score -
# Average Reading Score -
# % Passing Math-
# % Passing Reading-
# % Overall Passing (The percentage of students that passed math and reading.)-

### total students
#pulls in the student per school into the summary dataframe
sch_students=school_and_student_df.groupby('school_name').count()
sch_summary_df=pd.DataFrame(sch_students['student_name'])
#renames the header to Total Students
sch_summary_df = sch_summary_df.rename(columns={'student_name':'Total Students'})

##school type
# - this code pulls in the type and budget column from the original school data
sch_summary_df=pd.merge(sch_summary_df,school_data[['school_name','type','budget']],on="school_name",how="left")
#resets the index, previous line ---- maybe I could use ignore_index=True in the previous line to eliminate the need for this line
sch_summary_df=sch_summary_df.set_index('school_name')

#### budget per student
sch_summary_df["Budget Per Student"]=sch_summary_df["budget"]/sch_summary_df["Total Students"]

####calculates the averages for all series
sch_avgs_gbo = school_and_student_df.groupby("school_name").mean()
# sch_avgs_gbo.head()
#pulls in the average reading score to the summary df
sch_summary_df['Average Reading Score']=sch_avgs_gbo['reading_score']
#pulls in the average math score to the summary df
sch_summary_df['Average Math Score']=sch_avgs_gbo['math_score']


### pulls in the number of students passing reading to the summary df
sch_passing_reading = school_and_student_df.loc[school_and_student_df['reading_score']>=70,['school_name','Student ID']]
sch_passing_reading_gbo=sch_passing_reading.groupby("school_name").count()
sch_summary_df=pd.merge(sch_summary_df,sch_passing_reading_gbo,how='outer',on="school_name")
sch_summary_df=sch_summary_df.rename(columns={"Student ID":"Students Passing Reading"})

### pulls in the number of stuents passing math to the summary df
sch_passing_math=school_and_student_df.loc[school_and_student_df['math_score']>=70,['school_name','Student ID']]
sch_passing_math_gbo=sch_passing_math.groupby("school_name").count()
sch_summary_df=pd.merge(sch_summary_df,sch_passing_math_gbo,how='outer',on="school_name")
sch_summary_df=sch_summary_df.rename(columns={"Student ID":"Students Passing Math"})

### pulls in the number of students passing math and reading 
sch_passing_both=school_and_student_df.loc[((school_and_student_df['math_score']>=70) & (school_and_student_df['reading_score']>=70)) ,['school_name','Student ID']]
sch_passing_both_gbo=sch_passing_both.groupby("school_name").count()
sch_summary_df=pd.merge(sch_summary_df,sch_passing_both_gbo,how='outer',on="school_name")
sch_summary_df=sch_summary_df.rename(columns={"Student ID":"Students Passing Both"})

######calculates column for percentages (math,reading,overall) 
sch_percent_listy_loop=['Reading','Math','Both']
for category in sch_percent_listy_loop:
    sch_summary_df["% Passing "+category]=sch_summary_df["Students Passing "+category]/sch_summary_df["Total Students"]

###removes the # of students passing math reading and both since it isn't requested, just needed it for the % calcs
sch_del_list=[]
for category in sch_percent_listy_loop:
    sch_del_list.append("Students Passing "+category)

for deleter in sch_del_list:
    del sch_summary_df[deleter]

##-----------------------------------------------------------------------------------------
####formatting
#makes the lower case titles into titlecase and renames budget to Total Budget, could have used the titlecase function
sch_summary_df=sch_summary_df.rename(columns={'type':'Type','budget':'Total Budget'})

#reorders the columns in the requested order
sch_summary_df=sch_summary_df[['Type','Total Students', 'Total Budget', 'Budget Per Student', 'Average Math Score','Average Reading Score', '% Passing Math','% Passing Reading', '% Passing Both']]


####formating values
#formats the total budget and budget per student to currency format
to_currency_format =['Total Budget',"Budget Per Student"]
for col in to_currency_format:
    sch_summary_df[col]=sch_summary_df[col].map("${:,.2f}".format)

#percent formatting
for perc in sch_percent_listy_loop:
    sch_summary_df["% Passing "+perc]=sch_summary_df["% Passing "+perc].map('{:.2%}'.format)




sch_summary_df

,Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [1003]:

#sorts the summary df by % passing both in descending order
top_schools_df=sch_summary_df.sort_values('% Passing Both',ascending=False)
#simplifies the df to only include the first 5 entries
top_schools_df=top_schools_df.iloc[0:5,0:len(top_schools_df.columns)]
top_schools_df

,Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [1004]:


#sorts the summary df by % passing both in ascending order
bottom_schools_df=sch_summary_df.sort_values('% Passing Both',ascending=True)
#simplifies the df to only include the first 5 entries
bottom_schools_df=bottom_schools_df.iloc[0:5,0:len(bottom_schools_df.columns)]
bottom_schools_df





,Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [1005]:

#creates a dataframe for each grade
grade_9th = school_and_student_df.loc[school_and_student_df['grade']=="9th",:]
grade_10th = school_and_student_df.loc[school_and_student_df['grade']=="10th",:]
grade_11th = school_and_student_df.loc[school_and_student_df['grade']=="11th",:]
grade_12th = school_and_student_df.loc[school_and_student_df['grade']=="12th",:]

#creates a gbo for each df calculating the mean
grade_9th_math_scores = grade_9th.groupby(['school_name'])['math_score'].mean()
grade_10th_math_scores = grade_10th.groupby(['school_name'])['math_score'].mean()
grade_11th_math_scores = grade_11th.groupby(['school_name'])['math_score'].mean()
grade_12th_math_scores = grade_12th.groupby(['school_name'])['math_score'].mean()

#merges 9th and 10th grade
math_summary_df=pd.merge(grade_9th_math_scores,grade_10th_math_scores, on="school_name",suffixes=("9th Grade","10th Grade"))
math_summary_df=math_summary_df.rename(columns={"math_score9th Grade":"9th Grade Math Average","math_score10th Grade":"10th Grade Math Average"})

#merges 11th grade
math_summary_df=pd.merge(math_summary_df,grade_11th_math_scores, on="school_name")
math_summary_df=math_summary_df.rename(columns={"math_score":"11th Grade Math Average"})
#merges 12th grade
math_summary_df=pd.merge(math_summary_df,grade_12th_math_scores, on="school_name")
math_summary_df=math_summary_df.rename(columns={"math_score":"12th Grade Math Average"})

math_summary_df


,9th Grade Math Average,10th Grade Math Average,11th Grade Math Average,12th Grade Math Average
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [1006]:

#creates a gbo for each df calculating the mean
grade_9th_read_scores = grade_9th.groupby(['school_name'])['reading_score'].mean()
grade_10th_read_scores = grade_10th.groupby(['school_name'])['reading_score'].mean()
grade_11th_read_scores = grade_11th.groupby(['school_name'])['reading_score'].mean()
grade_12th_read_scores = grade_12th.groupby(['school_name'])['reading_score'].mean()


#merges 9th and 10th grade
read_summary_df=pd.merge(grade_9th_read_scores,grade_10th_read_scores, on="school_name",suffixes=("9th Grade","10th Grade"))
read_summary_df=read_summary_df.rename(columns={"reading_score9th Grade":"9th Grade Reading Average","reading_score10th Grade":"10th Grade Reading Average"})

#merges 11th grade
read_summary_df=pd.merge(read_summary_df,grade_11th_read_scores, on="school_name")
read_summary_df=read_summary_df.rename(columns={"reading_score":"11th Grade Reading Average"})
#merges 12th grade
read_summary_df=pd.merge(read_summary_df,grade_12th_read_scores, on="school_name")
read_summary_df=read_summary_df.rename(columns={"reading_score":"12th Grade Reading Average"})

read_summary_df


,9th Grade Reading Average,10th Grade Reading Average,11th Grade Reading Average,12th Grade Reading Average
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [1007]:
#start from og
### calculate the spending per student at each school
#calcs the budget per school
score_by_spending = school_data[['school_name','budget']]
score_by_spending=score_by_spending.set_index("school_name")

#calc the students per school
stu_per_school=sch_students["student_name"]
score_by_spending=pd.merge(score_by_spending,stu_per_school,how="outer",on="school_name")
score_by_spending["Budget Per Student"]=score_by_spending["budget"]/score_by_spending["student_name"]

#calculating the bins based on the data
bins=[]
x_range=score_by_spending["Budget Per Student"].max()-score_by_spending["Budget Per Student"].min()
x_high=score_by_spending["Budget Per Student"].max()
x_low=score_by_spending["Budget Per Student"].min()
x_range_per_bin=x_range/3
bins.append(round(x_low,0))
bins.append(round(x_low + x_range_per_bin,0))
bins.append(round(x_low + x_range_per_bin + x_range_per_bin,0))
bins.append(round(x_low + x_range_per_bin + x_range_per_bin +x_range_per_bin,0))
#seeting the bin labeles
bin_labels=["Low","Mid","High"]
# adding a column with the binned info
score_by_spending["Spending Per Student Ranks"]=pd.cut(score_by_spending["Budget Per Student"],bins,labels=bin_labels,include_lowest=True)

# pulls in the average math score, average reading score, % passing math, % passing reading and % passing overall
pull_from_school_summary_list=['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Passing Both']
for blerp in pull_from_school_summary_list:
    score_by_spending=pd.merge(score_by_spending,sch_summary_df[[blerp]],how='left',on='school_name')
#renames the student_name field to # of students
score_by_spending=score_by_spending.rename(columns={"student_name":"Number of Students"})


# reordering the columns
score_by_spending=score_by_spending[['Spending Per Student Ranks','budget', 'Number of Students', 'Budget Per Student', 'Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading','% Passing Both']]


score_by_spending=score_by_spending.sort_values('Budget Per Student',ascending=False)


score_by_spending


,Spending Per Student Ranks,budget,Number of Students,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Huang High School,High,1910635,2917,655.0,76.629414,81.182722,65.68%,81.32%,53.51%
Hernandez High School,High,3022020,4635,652.0,77.289752,80.934412,66.75%,80.86%,53.53%
Johnson High School,High,3094650,4761,650.0,77.072464,80.966394,66.06%,81.22%,53.54%
Ford High School,High,1763916,2739,644.0,77.102592,80.746258,68.31%,79.30%,54.29%
Figueroa High School,High,1884411,2949,639.0,76.711767,81.158020,65.99%,80.74%,53.20%
Thomas High School,High,1043130,1635,638.0,83.418349,83.848930,93.27%,97.31%,90.95%
Rodriguez High School,High,2547363,3999,637.0,76.842711,80.744686,66.37%,80.22%,52.99%
Bailey High School,Mid,3124928,4976,628.0,77.048432,81.033963,66.68%,81.93%,54.64%
Griffin High School,Mid,917500,1468,625.0,83.351499,83.816757,93.39%,97.14%,90.60%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [1008]:
#start from og
### 
#pulls the size and school name from original doc
score_by_size = school_data[['school_name','size']]
score_by_size=score_by_size.set_index("school_name")


#calculates the bins sizes
y_bins=[]
y_high=score_by_size['size'].max()
y_low=score_by_size['size'].min()
y_range=y_high-y_low
y_range_per_bin=y_range/3

y_bins.append(round(y_low,0))
y_bins.append(round(y_low+y_range_per_bin,0))
y_bins.append(round(y_low+y_range_per_bin+y_range_per_bin,0))
y_bins.append(round(y_low+y_range_per_bin+y_range_per_bin+y_range_per_bin,0))

#sets the bins labels
y_bins_labels= ["Small","Medium","Large"]

#does the 'binning" in the socre_by_size df
score_by_size["School Size Rank"]=pd.cut(score_by_size["size"],y_bins,labels=y_bins_labels,include_lowest=True)

#pulls in the requested fields from the sch_summary ** see previous section for pull_from_school_summary_list
for blerp in pull_from_school_summary_list:
    score_by_size=pd.merge(score_by_size,sch_summary_df[[blerp]],how='left',on='school_name')

# #reorders the columns
score_by_size=score_by_size[['School Size Rank','size','Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading','% Passing Both']]


#sorts the column by size
score_by_size=score_by_size.sort_values('size',ascending=False)

score_by_size

,School Size Rank,size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,
Bailey High School,Large,4976,77.048432,81.033963,66.68%,81.93%,54.64%
Johnson High School,Large,4761,77.072464,80.966394,66.06%,81.22%,53.54%
Hernandez High School,Large,4635,77.289752,80.934412,66.75%,80.86%,53.53%
Rodriguez High School,Large,3999,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,Medium,2949,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,Medium,2917,76.629414,81.182722,65.68%,81.32%,53.51%
Ford High School,Medium,2739,77.102592,80.746258,68.31%,79.30%,54.29%
Wilson High School,Medium,2283,83.274201,83.989488,93.87%,96.54%,90.58%
Cabrera High School,Small,1858,83.061895,83.975780,94.13%,97.04%,91.33%


## Scores by School Type

* Perform the same operations as above, based on school type

In [1009]:
dist_chart_score=score_by_size

del dist_chart_score["School Size Rank"]
del dist_chart_score["size"]

indi_school_data=school_data.set_index("school_name")
#pulls in the type to the summary frame
dist_chart_score=pd.merge(dist_chart_score,indi_school_data[["type"]],how='left',on='school_name')

dist_chart_score=dist_chart_score[['type','Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading', '% Passing Both']]

dist_chart_score

,type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,
Bailey High School,District,77.048432,81.033963,66.68%,81.93%,54.64%
Johnson High School,District,77.072464,80.966394,66.06%,81.22%,53.54%
Hernandez High School,District,77.289752,80.934412,66.75%,80.86%,53.53%
Rodriguez High School,District,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,District,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,District,76.629414,81.182722,65.68%,81.32%,53.51%
Ford High School,District,77.102592,80.746258,68.31%,79.30%,54.29%
Wilson High School,Charter,83.274201,83.989488,93.87%,96.54%,90.58%
Cabrera High School,Charter,83.061895,83.975780,94.13%,97.04%,91.33%
